In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer #for missing values
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder #for categorical data(ordinal)
from sklearn.preprocessing import MinMaxScaler #for scalling
from sklearn.tree import DecisionTreeClassifier #for making predictions

In [7]:
df = pd.read_csv('hotel_booking_data.csv')

In [8]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [9]:
df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

## customer type,deposit_type,days_in_waiting_list,is_repeated_guest,previous_bookings_not_canceled,hotel,is_cancelled

In [10]:
new_df=df[['customer_type','deposit_type','days_in_waiting_list','is_repeated_guest','previous_bookings_not_canceled','hotel','is_canceled']]

In [36]:
new_df['hotel'].shape

(119390,)

In [12]:
df['deposit_type'].value_counts()

deposit_type
No Deposit    104641
Non Refund     14587
Refundable       162
Name: count, dtype: int64

In [13]:
#train_test_split
x_train,x_test,y_train,y_test = train_test_split(new_df.drop(columns=['is_canceled']),
                                                 new_df['is_canceled'],
                                                test_size=0.2,
                                                )

### As there are no missing values,I am not going to use SimpleImputer

In [14]:
#new_df.isnull().sum() 

In [15]:
from sklearn.preprocessing import OrdinalEncoder

In [16]:
#OneHotEncoding(ordinal encoding)
oe = OrdinalEncoder(categories=[['No Deposit','Non Refund','Refundable']])
x_train_deposit = oe.fit_transform(x_train[['deposit_type']])
x_test_deposit = oe.transform(x_test[['deposit_type']])


In [45]:
#OneHotEncoding(nominal encodingI
ohe = OneHotEncoder(sparse_output=False)
x_train_customer_hotel = ohe.fit_transform(x_train[['customer_type','hotel']])
x_test_customer_hotel = ohe.transform(x_test[['customer_type','hotel']])

In [47]:
x_train_rem = x_train.drop(columns=['deposit_type','customer_type','hotel'])
x_test_rem = x_test.drop(columns=['deposit_type','customer_type','hotel'])

In [48]:
x_train_transformed = np.concatenate((x_train_rem,x_train_deposit,x_train_customer_hotel),axis=1)


In [49]:
x_test_transformed = np.concatenate((x_test_rem,x_test_deposit,x_test_customer_hotel),axis=1)

In [51]:
x_test_transformed.shape

(23878, 10)

In [52]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [55]:
y_pred = clf.predict(x_test_transformed)
y_pred

array([0, 1, 0, ..., 0, 0, 0])

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7514448446268531

# now,let suppose we deploy it on website...where anyone giving the informations,can see the prediction whether the customer 
# will cancelled the booking or not

In [57]:
import pickle